In [1]:
%load_ext watermark
%watermark -p torch,joblib,numpy,eli5,jieba,sentencepiece -m

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


torch 0.4.1
joblib 0.12.2
numpy 1.15.0
eli5 0.8
jieba 0.39
sentencepiece n

compiler   : GCC 7.2.0
system     : Linux
release    : 4.10.0-32-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from pathlib import Path
import re

import torch
import numpy as np
import joblib
import sentencepiece as spm
import jieba

In [3]:
path = Path("../data/cache/lm_bpe_douban/")
model = torch.load(path / "sentiment_model.pth")
sp = spm.SentencePieceProcessor()
sp.Load("../data/bpe_model.model")

True

In [4]:
model = model.cpu()
model.reset()
model = model.eval()

In [5]:
UNK = 0
BEG = 1
def get_prediction(texts):
    input_tensor = torch.from_numpy(np.array([1] + sp.EncodeAsIds(texts))).long().unsqueeze(1)
    return model(input_tensor)[0].data.cpu().numpy()[0, 0]

In [6]:
get_prediction("看了快一半了才发现是mini的广告")

3.5602758

## ELI5 (LIME)

In [7]:
UNK = 2
def get_proba(rows):
    """eli5 only supports classifier, so we need to do some transformations."""
    probs = []
    for texts in rows:
        model.reset()
        texts = re.sub(r"\s+", " ", texts)
        texts.replace("qqq", " ")
        input_tensor = torch.from_numpy(
            np.array([1] + sp.EncodeAsIds(texts))
        ).long().unsqueeze(1)
        pos = (np.clip(model(input_tensor)[0].data.cpu().numpy()[0, 0], 1, 5) - 1) / 4
        probs.append(np.array([1-pos, pos]))
    return np.stack(probs, axis=0)

### Example 1

In [8]:
get_proba(["看了快一半了才发现是mini的广告"])

array([[0.35993, 0.64007]])

In [9]:
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut("看了快一半了才发现是mini的广告", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.555 seconds.
Prefix dict has been built succesfully.


In [10]:
te.metrics_

{'mean_KL_divergence': 0.012447314202164288, 'score': 0.7746055119915863}

In [11]:
te.samples_[:10]

['看 了  一半  才 发现 是 mini 的 ',
 ' 了 快     是   ',
 '   一半  才  是  的 ',
 '看 了  一半  才  是 mini 的 广告',
 '看   一半  才  是   广告',
 '看  快 一半 了   是 mini  广告',
 '          ',
 ' 了 快  了 才 发现  mini  ',
 '          ',
 '看 了   了  发现   的 广告']

#### Character-based Whitebox

In [12]:
te = TextExplainer(random_state=42, n_samples=5000, char_based=True)
te.fit("看了快一半了才发现是mini的广告", get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [13]:
te.metrics_

{'mean_KL_divergence': 0.004799421089570338, 'score': 0.984549507132481}

In [14]:
te.samples_[:10]

['了一半发是ni广告',
 '看一半了才是min广',
 'm',
 '看了一半了现ii的广告',
 '了快是i的广',
 '看快半了才现是ini的告',
 '广',
 '看了快一半了才的广告',
 '了快一半了发现是mn的广',
 '看半了mn广']

### Example 2

In [15]:
te = TextExplainer(random_state=42, n_samples=2000)
te.fit(" ".join(jieba.cut("妈蛋，简直太好看了。最后的 DJqqqbattle 部分，兴奋的我，简直想从座位上站起来一起扭", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [16]:
te.samples_[:10]

['妈蛋 ， 简直 太 好看 了 。 最后 的      部分 ， 兴奋  我 ， 简直   座位  站 起来  ',
 '妈蛋 ， 简直 太 好看 了 。  的   DJqqqbattle   部分 ，  的 我 ， 简直  从 座位 上 站 起来 一起 扭',
 ' ， 简直    。 最后        ， 兴奋 的  ，    座位 上    扭',
 '妈蛋 ， 简直  好看  。  的   DJqqqbattle   部分 ， 兴奋 的  ， 简直    上  起来  ',
 ' ， 简直 太   。     DJqqqbattle    ，  的  ，    座位     ',
 '妈蛋 ，  太   。 最后        ， 兴奋   ，  想       ',
 '妈蛋 ， 简直   了 。 最后       部分 ， 兴奋 的 我 ， 简直 想    站   ',
 ' ， 简直  好看  。 最后 的       ，  的  ， 简直     站   扭',
 ' ， 简直 太 好看 了 。 最后 的       ， 兴奋 的 我 ，   从 座位 上 站 起来 一起 ',
 '妈蛋 ，   好看 了 。         ， 兴奋   ， 简直 想    站  一起 扭']

In [17]:
te.metrics_

{'mean_KL_divergence': 0.008596786731327001, 'score': 0.9581006431253122}

### Example 3

In [18]:
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut("十亿元很难花掉吗？投资一部《阿修罗》，瞬间就赔光了啊。这设定太没说服力了。", cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [19]:
get_proba([" ".join(jieba.cut("十亿元很难花掉吗？投资一部《阿修罗》，瞬间就赔光了啊。这设定太没说服力了。", cut_all=False))])

array([[0.74417, 0.25583]])

In [20]:
te.metrics_

{'mean_KL_divergence': 0.004375076827941791, 'score': 0.954368466661393}

### Example 4

In [21]:
texts = (
    "爱上了让雷诺。爱上了这个意大利杀手的温情。爱上了他脸部轮廓，和刚毅的线条。"
    "“人生诸多辛苦，是不是只有童年如此。”玛蒂尔达问。里昂说，“一直如此。”这样的话，击中了内心深处。"
)
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [22]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.21405, 0.78595]])

In [23]:
te.metrics_

{'mean_KL_divergence': 0.007109762064195683, 'score': 0.9962618614341539}

### Example 5

In [24]:
texts = "看到泪飙，世上最温情的杀手。 幸运的姑娘在12岁就遇到了真正的男人。"
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [25]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.12378, 0.87622]])

In [26]:
te.metrics_

{'mean_KL_divergence': 0.006987204622608167, 'score': 0.9943648144781647}

### Example 6

In [27]:
texts = (
    "当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，"
    "这部电影在越来越多的人们心中的地位已超越了《阿甘》。每当现实令我疲惫得产生无力感，"
    "翻出这张碟，就重获力量。毫无疑问，本片位列男人必看的电影前三名！回顾那一段经典台词："
    "“有的人的羽翼是如此光辉，即使世界上最黑暗的牢狱，也无法长久地将他围困！”"
)
te = TextExplainer(random_state=42, n_samples=5000)
te.fit(" ".join(jieba.cut(texts, cut_all=False)), get_proba)
te.show_prediction(target_names=["neg", "pos"])

In [28]:
get_proba([" ".join(jieba.cut(texts, cut_all=False))])

array([[0.19769, 0.80231]])

In [29]:
te.metrics_

{'mean_KL_divergence': 0.00821021575734618, 'score': 0.9952506919053602}